In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
import pandas as pd
import geopandas as gpd
import pydeck as pdk
import datetime
import altair as alt


st.set_page_config(layout="wide")
session = get_active_session()

snow_df = pd.read_csv('seoul_real_estate_selected_merged_20250725_141605.csv')
total_list = ['전체']
gu_list = sorted(snow_df['CGG_NM'].unique())
gu_list = total_list + gu_list
selected_gu = st.multiselect('관심 있는 구를 선택하세요', gu_list)

hifives_val = st.slider(
  "선택 년도",
  min_value=2006,
  max_value=datetime.datetime.now().year,
  value=(2009, 2015),
)

@st.cache_data()
def load_data(snow_df, year, selected_gu):
    min_year = year[0]
    max_year = year[1]
    if '전체' not in selected_gu : 
        snow_df = snow_df[snow_df['CGG_NM'].isin(selected_gu)]
    snow_df = snow_df[(snow_df['RCPT_YR'] >= min_year) & (snow_df['RCPT_YR'] <= max_year)]
    snow_df = snow_df.groupby(['CGG_NM', 'CGG_CD']).size().reset_index(name='CNT')
    return snow_df

df = load_data(snow_df, hifives_val, selected_gu )  # 슬라이더 값 전달
gdf = gpd.read_file('./seoul_gu_boundaries.geojson')
gdf = gdf.merge(df, left_on='name', right_on='CGG_NM', how='left').fillna(0)

# GeoJsonPolygons 컬러화
geojson = gdf.__geo_interface__

# 레이어 1: GeoJsonLayer 구별 채움색 (조건부)
layer_polygons = pdk.Layer(
    'GeoJsonLayer',
    data=geojson,
    stroked=True,
    filled=True,
    extruded=False,
    get_fill_color='properties.CNT > 0 ? [255, 165, 0, 150] : [200, 200, 200, 50]',
    get_line_color=[255, 255, 255],
    pickable=True,
)

diff_year = hifives_val[1] - hifives_val[0]
if diff_year == 0 : 
    diff_year = 1
    
# 레이어 2: 중심점 ScatterplotLayer
centroids = gdf.copy()
centroids['lon'] = centroids.geometry.centroid.x
centroids['lat'] = centroids.geometry.centroid.y
centroids['radius'] = centroids['CNT'] * 0.2 / diff_year

layer_points = pdk.Layer(
    'ScatterplotLayer',
    data=centroids[centroids['CNT'] > 0],
    get_position=['lon', 'lat'],
    get_radius='radius',  # 비율 조절
    get_fill_color=[30, 144, 255],
    pickable=True,
    opacity=0.8,
)

view_state = pdk.ViewState(latitude=37.56, longitude=126.98, zoom=11)

st.subheader("각 구별 부동산 거래량")
st.pydeck_chart(pdk.Deck(layers=[layer_polygons, layer_points], initial_view_state=view_state, tooltip={"text":"{name} \n총 거래량: {CNT}"}))

st.subheader("거래량 테이블")
st.dataframe(df, use_container_width=True)

df_barchart = snow_df[snow_df['CGG_NM'].isin(selected_gu)]

view_dong = st.multiselect('각 구에 해당하는 동별 부동산 거래량', selected_gu)

if view_dong:
    # 1. 선택된 구에 해당하는 데이터 필터링
    filtered_df = snow_df[
        (snow_df['CGG_NM'].isin(view_dong)) &
        (snow_df['RCPT_YR'] >= hifives_val[0]) &
        (snow_df['RCPT_YR'] <= hifives_val[1])
    ]

    # 2. CGG_NM(구), STDG_NM(동) 기준으로 거래 건수 집계
    grouped_df = (
        filtered_df.groupby(['CGG_NM', 'STDG_NM'])
        .size()
        .reset_index(name='CNT')
    )

    # 3. 바 차트로 시각화
    st.subheader("선택된 구의 동별 부동산 거래량 (Bar Chart)")

    chart = (
        alt.Chart(grouped_df)
        .mark_bar()
        .encode(
            x=alt.X('STDG_NM:N', sort='-y', title='법정동'),
            y=alt.Y('CNT:Q', title='거래량'),
            color='CGG_NM:N',
            tooltip=['CGG_NM', 'STDG_NM', 'CNT']
        )
        .properties(width=800, height=400)
    )

    st.altair_chart(chart, use_container_width=True)

else:
    st.info("동별 거래량을 보려면 구를 하나 이상 선택하세요.")